In [602]:
# open scores.csv

import matplotlib.pyplot as plt
import pandas as pd

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import ConnectionPatch
from matplotlib.patches import Rectangle


df = pd.read_csv('scores.csv')
df.head(5)

,mu,test_set_size,seed,bootstrap_bca_5_score,bootstrap_bca_last_score,bootstrap_5_score,bootstrap_last_score,bootstrap_analytical_score,clt_slutsky_score,t_distribution_score
0,0.85,15,1998,0,0,0,0,0,0,0
1,0.85,1000,1998,0,0,0,0,0,0,0
2,0.55,8,5,0,0,0,0,0,0,0
3,0.55,8,0,0,0,0,0,0,0,0
4,0.55,8,63,0,0,0,0,0,0,0


In [603]:
# Remove first two rows
df = df.iloc[2:]

In [604]:
# remove test_set_size == 5000
df = df[df.test_set_size != 5000]

In [605]:
# Display the different mu values, and the different test_set_size values
mus: list[float] = sorted(list(df['mu'].unique()))
test_set_sizes: list[int] = sorted(list(df['test_set_size'].unique()))

print(mus)
print(test_set_sizes)

[0.45, 0.5, 0.55, 0.8, 0.92, 0.925, 0.93, 0.9925]
[8, 10, 20, 30, 40, 50, 75, 100, 150, 200, 350, 500, 750, 1000]


In [606]:
# Average coverage score for the whole dataframe
for col_name in [
    "bootstrap_bca_5_score",
    "bootstrap_bca_last_score",
    "bootstrap_5_score",
    "bootstrap_last_score",
    "bootstrap_analytical_score",
    "clt_slutsky_score",
    "t_distribution_score",
]:
    print(
        col_name, 
        ":",
        round(
            df[col_name].value_counts()[0]/len(df),
            5
        )
    )

print("---")

# Average below and above coverage score for the whole dataframe
for col_name in [
    "bootstrap_bca_5_score",
    "bootstrap_bca_last_score",
    "bootstrap_5_score",
    "bootstrap_last_score",
    "bootstrap_analytical_score",
    "clt_slutsky_score",
    "t_distribution_score",
]:
    print(
        col_name, 
        ":",
        round(
            df[col_name].value_counts()[-1]/len(df),
            5
        ),
        "and",
        round(
            df[col_name].value_counts()[1]/len(df),
            5
        )
    )

bootstrap_bca_5_score : 0.84412
bootstrap_bca_last_score : 0.86566
bootstrap_5_score : 0.85198
bootstrap_last_score : 0.86608
bootstrap_analytical_score : 0.86698
clt_slutsky_score : 0.85894
t_distribution_score : 0.86734
---
bootstrap_bca_5_score : 0.02388 and 0.132
bootstrap_bca_last_score : 0.0102 and 0.12414
bootstrap_5_score : 0.12708 and 0.02094
bootstrap_last_score : 0.11958 and 0.01434
bootstrap_analytical_score : 0.11916 and 0.01386
clt_slutsky_score : 0.12684 and 0.01422
t_distribution_score : 0.12096 and 0.0117


In [607]:
df_0dot45 = df[df['mu'] == 0.45]
df_0dot5 = df[df['mu'] == 0.5]
df_0dot55 = df[df['mu'] == 0.55]
df_0dot8 = df[df['mu'] == 0.8]
df_0dot92 = df[df['mu'] == 0.92]
df_0dot925 = df[df['mu'] == 0.925]
df_0dot93 = df[df['mu'] == 0.93]
df_0dot9925 = df[df['mu'] == 0.9925]

In [608]:
len(df_0dot45), len(df_0dot5), len(df_0dot55), len(df_0dot8), len(df_0dot92), len(df_0dot925), len(df_0dot93), len(df_0dot9925)

(2076, 2077, 2120, 2081, 2073, 2080, 2080, 2080)

In [609]:
len(df_0dot45)//14, len(df_0dot5)//14, len(df_0dot55)//14, len(df_0dot8)//14, len(df_0dot92)//14, len(df_0dot925)//14, len(df_0dot93)//14, len(df_0dot9925)//14

(148, 148, 151, 148, 148, 148, 148, 148)

In [610]:
for df_filtered in [
    df_0dot45,
    df_0dot5,
    df_0dot55,
    df_0dot8,
    df_0dot92,
    df_0dot925,
    df_0dot93,
    df_0dot9925,
]:
    # Small test set sizes and big test set sizes
    df_small = df_filtered[df_filtered['test_set_size'] < 100]
    df_big = df_filtered[df_filtered['test_set_size'] >= 100]

    for df_sub in [df_small, df_big]:
        print('mu =', df_sub['mu'].iloc[0])
        print("\tSmall test set sizes" if (df_sub['test_set_size'] < 100).all() else "Big test set sizes")

        # Best coverage is 0.95
        # What method has the best coverage
        print(f"\t\tCoverage")
        best_method = None
        best_delta_coverage = 1
        for col_name in [
            "bootstrap_bca_5_score",
            "bootstrap_bca_last_score",
            "bootstrap_5_score",
            "bootstrap_last_score",
            "bootstrap_analytical_score",
            "clt_slutsky_score",
            "t_distribution_score",
        ]: 
            coverage = df_sub[col_name].value_counts()[0]/len(df_sub) if 0 in df_sub[col_name].value_counts() else 0
            if abs(coverage-0.95) < best_delta_coverage:
                best_delta_coverage = abs(coverage-0.95)
                best_coverage = coverage
                best_method = col_name
            print(f"\t\t\t{col_name} coverage: {round(coverage, 5)}")

        print(f"\t\tBest coverage: {round(best_coverage, 5)} with method {best_method}")
        print("---")

        # What method is the most balanced, ideally 0.025 on each side
        print(f"\t\tBalance")
        best_method = None
        best_delta_coverage = 1
        for col_name in [
            "bootstrap_bca_5_score",
            "bootstrap_bca_last_score",
            "bootstrap_5_score",
            "bootstrap_last_score",
            "bootstrap_analytical_score",
            "clt_slutsky_score",
            "t_distribution_score",
        ]:
            print(
                f"\t\t\t{col_name} below: ",
                below := round(
                    (df_sub[col_name].value_counts()[-1]/len(df_sub) if -1 in df_sub[col_name].value_counts() else 0),
                    5
                )
            )
            print(
                f"\t\t\t{col_name} above: ",
                above := round( 
                    (df_sub[col_name].value_counts()[1]/len(df_sub) if 1 in df_sub[col_name].value_counts() else 0),
                    5
                )
            )

            if abs(below-above) < best_delta_coverage:
                best_delta_coverage = abs(below-above)
                best_method = col_name
        
        print(f"\t\tBest balance: {round(best_delta_coverage, 5)} with method {best_method}")


        print(" --- ")
        print(" --- ")
        print(" --- ")

mu = 0.45
	Small test set sizes
		Coverage
			bootstrap_bca_5_score coverage: 0.92404
			bootstrap_bca_last_score coverage: 0.94281
			bootstrap_5_score coverage: 0.93655
			bootstrap_last_score coverage: 0.95085
			bootstrap_analytical_score coverage: 0.94996
			clt_slutsky_score coverage: 0.94906
			t_distribution_score coverage: 0.96247
		Best coverage: 0.94996 with method bootstrap_analytical_score
---
		Balance
			bootstrap_bca_5_score below:  0.0277
			bootstrap_bca_5_score above:  0.04826
			bootstrap_bca_last_score below:  0.02055
			bootstrap_bca_last_score above:  0.03664
			bootstrap_5_score below:  0.03753
			bootstrap_5_score above:  0.02592
			bootstrap_last_score below:  0.02949
			bootstrap_last_score above:  0.01966
			bootstrap_analytical_score below:  0.03038
			bootstrap_analytical_score above:  0.01966
			clt_slutsky_score below:  0.03038
			clt_slutsky_score above:  0.02055
			t_distribution_score below:  0.02502
			t_distribution_score above:  0.01251
		Best bala

Figures

In [611]:
for df_filtered in [df_0dot45, df_0dot8, df_0dot925, df_0dot9925]:
    for col_name, method_name in zip(
        [
            "bootstrap_bca_5_score",
            "bootstrap_bca_last_score",
            "bootstrap_5_score",
            "bootstrap_last_score",
            "bootstrap_analytical_score",
            "clt_slutsky_score",
            "t_distribution_score",
        ],
        [
            "Bias-Corrected and Accelerated\nBootstrap (B = 100)",
            "Bias-Corrected and Accelerated\nBootstrap (B = 5000)",
            "Percentile Bootstrap\n(B = 100)",
            "Percentile Bootstrap\n(B = 5000)",
            "Percentile Bootstrap\n(Analytical)",
            "Central Limit Theorem\n& Slutsky",
            "Student's\nt-distribution"
        ]
    ):
        # Plot coverage of each method as a function of test set size
        coverages: list[float] = []
        for test_set_size in test_set_sizes:
            df_filtered_test_set_size = df_filtered[df_filtered['test_set_size'] == test_set_size]
            number_ins: int = df_filtered_test_set_size[col_name].value_counts()[0] if 0 in df_filtered_test_set_size[col_name].value_counts() else 0
            total: int = len(df_filtered_test_set_size)
            coverage: float = number_ins/total
            coverages.append(coverage)

        my_linewidth_adaptation: float = 0.3 if (
            method_name == "Student's\nt-distribution") else 0.7 if (
                method_name == "Percentile Bootstrap\n(Analytical)"
            ) else 0
        plt.plot(
            test_set_sizes,
            coverages,
            label=method_name,
            alpha=1-my_linewidth_adaptation,
            linewidth=my_linewidth_adaptation*5+1,
            linestyle='--' if (
                method_name in ["Bias-Corrected and Accelerated\nBootstrap (B = 5000)", "Percentile Bootstrap\n(B = 5000)"]) else (
                    '-.' if method_name == "Central Limit Theorem\n& Slutsky" else '-'
            )
        )

    # Legend right (outside of plot)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    plt.ylim(0.6, 1.03)
    plt.axhline(y=0.95, color='black', linestyle='dotted', alpha=0.8)
    plt.title(r'$\mu$' + f" = {df_filtered['mu'].iloc[0]:.4f}")
    
    # xlabel and ylabel
    plt.xlabel("Test set size")
    plt.ylabel("Coverage")

    # Rectangle
    ax = plt.gca()  # Get the current axis
    axins = inset_axes(ax, width="45%", height="50%", loc="lower right", borderpad=2.5)
    
    x1, x2, y1, y2 = 800, 1000, 0.9, 1.01  # Define the region you want to zoom into
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(rect)

    # Create connection lines between rectangle and inset plot
    xy1 = (x1, y1)
    xy1p = (x1, y2)
    xy4 = (x2, y1)
    xy4p = (x2, y2)
    con1 = ConnectionPatch(xyA=xy1, xyB=xy1p, coordsA="data", coordsB="data", axesA=ax, axesB=axins, color="black", linestyle='--', alpha=0.8)
    con4 = ConnectionPatch(xyA=xy4, xyB=xy4p, coordsA="data", coordsB="data", axesA=ax, axesB=axins, color="black", linestyle='--', alpha=0.8)
    ax.add_artist(con1)
    ax.add_artist(con4)
    
    # Inset (zoom) plot
    for col_name, method_name in zip(
        [
            "bootstrap_bca_5_score",
            "bootstrap_bca_last_score",
            "bootstrap_5_score",
            "bootstrap_last_score",
            "bootstrap_analytical_score",
            "clt_slutsky_score",
            "t_distribution_score",
        ],
        [
            "Bias-Corrected and Accelerated\nBootstrap (B = 100)",
            "Bias-Corrected and Accelerated\nBootstrap (B = 5000)",
            "Percentile Bootstrap\n(B = 100)",
            "Percentile Bootstrap\n(B = 5000)",
            "Percentile Bootstrap\n(Analytical)",
            "Central Limit Theorem\n& Slutsky",
            "Student's\nt-distribution"
        ]
    ):
        coverages: list[float] = []
        for test_set_size in test_set_sizes:
            df_filtered_test_set_size = df_filtered[df_filtered['test_set_size'] == test_set_size]
            number_ins: int = df_filtered_test_set_size[col_name].value_counts()[0] if 0 in df_filtered_test_set_size[col_name].value_counts() else 0
            total: int = len(df_filtered_test_set_size)
            coverage: float = number_ins/total
            coverages.append(coverage)

        my_linewidth_adaptation: float = 0.3 if (
            method_name == "Student's\nt-distribution") else 0.7 if (
                method_name == "Percentile Bootstrap\n(Analytical)"
            ) else 0
        axins.plot(
            test_set_sizes,
            coverages,
            label=method_name,
            alpha=1-my_linewidth_adaptation,
            linewidth=my_linewidth_adaptation*5+1,
            linestyle='--' if (
                method_name in ["Bias-Corrected and Accelerated\nBootstrap (B = 5000)", "Percentile Bootstrap\n(B = 5000)"]) else (
                    '-.' if method_name == "Central Limit Theorem\n& Slutsky" else '-'
            )
        )

    # Set the range for your zoomed-in area
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.axhline(y=0.95, color='black', linestyle='dotted', alpha=0.8)
    plt.title("Zoom")

    # Save as svg
    plt.savefig(f"coverage_{df_filtered['mu'].iloc[0]}.svg", bbox_inches='tight')
    plt.close()

ZeroDivisionError: division by zero

In [ ]:
for df_filtered in [df_0dot45, df_0dot8, df_0dot925, df_0dot9925]:
    plt.gca().set_prop_cycle(None)
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    color_index: int = 0
    for col_name, method_name in zip(
        [
            "bootstrap_bca_5_score",
            "bootstrap_bca_last_score",
            "bootstrap_5_score",
            "bootstrap_last_score",
            "bootstrap_analytical_score",
            "clt_slutsky_score",
            "t_distribution_score",
        ],
        [
            "Bias-Corrected and Accelerated\nBootstrap (B = 100)",
            "Bias-Corrected and Accelerated\nBootstrap (B = 5000)",
            "Percentile Bootstrap\n(B = 100)",
            "Percentile Bootstrap\n(B = 5000)",
            "Percentile Bootstrap\n(Analytical)",
            "Central Limit Theorem\n& Slutsky",
            "Student's\nt-distribution"
        ]
    ):
        # Plot coverage of each method as a function of test set size
        coverages_below: list[float] = []
        coverages_above: list[float] = []
        for test_set_size in test_set_sizes:
            df_filtered_test_set_size = df_filtered[df_filtered['test_set_size'] == test_set_size]
            number_ins_below: int = df_filtered_test_set_size[col_name].value_counts()[-1] if -1 in df_filtered_test_set_size[col_name].value_counts() else 0
            number_ins_above: int = df_filtered_test_set_size[col_name].value_counts()[1] if 1 in df_filtered_test_set_size[col_name].value_counts() else 0
            total: int = len(df_filtered_test_set_size)
            coverage_below: float = number_ins_below/total
            coverage_above: float = number_ins_above/total
            coverages_below.append(-coverage_below)
            coverages_above.append(coverage_above)
        
        # Get the next color from the color cycle
        current_color = color_cycle[color_index % len(color_cycle)]
        color_index += 1

        my_linewidth_adaptation: float = 0.3 if (
            method_name == "Student's\nt-distribution") else 0.7 if (
                method_name == "Percentile Bootstrap\n(Analytical)"
            ) else 0
        plt.plot(
            test_set_sizes,
            coverages_above,
            color=current_color,
            label=method_name,
            alpha=1-my_linewidth_adaptation,
            linewidth=my_linewidth_adaptation*5+1,
            linestyle='--' if (
                method_name in ["Bias-Corrected and Accelerated\nBootstrap (B = 5000)", "Percentile Bootstrap\n(B = 5000)"]) else (
                    '-.' if method_name == "Central Limit Theorem\n& Slutsky" else '-'
            )
        )
        plt.plot(
            test_set_sizes,
            coverages_below,
            color=current_color,
            alpha=1-my_linewidth_adaptation,
            linewidth=my_linewidth_adaptation*5+1,
            linestyle='--' if (
                method_name in ["Bias-Corrected and Accelerated\nBootstrap (B = 5000)", "Percentile Bootstrap\n(B = 5000)"]) else (
                    '-.' if method_name == "Central Limit Theorem\n& Slutsky" else '-'
            )
        )
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.ylim(-0.105, 0.105)
    plt.axhline(y=0.025, color='black', linestyle='dotted', alpha=0.8)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.8, linewidth=0.5)
    plt.axhline(y=-0.025, color='black', linestyle='dotted', alpha=0.8)
    plt.title(r'$\mu$' + f" = {df_filtered['mu'].iloc[0]:.4f}")
    
    # xlabel and ylabel
    plt.xlabel("Sample size")
    plt.ylabel("Relative frequency of the true parameter value\nfalling above or below the Confidence Interval")

    # Save as svg
    plt.savefig(f"balance_{df_filtered['mu'].iloc[0]}.svg", bbox_inches='tight')
    plt.close()